# Database Operations

This notebook provides interactive access to the Neon database using our CRUD operations.

## Setup

In [19]:
import asyncio
import nest_asyncio
import pandas as pd

from src.database import (
    get_db,
    get_podcast_by_id, 
    get_podcast_by_rss_url,
    create_podcast,
    list_podcasts,
    get_episode_by_guid,
    create_episode,
    get_podcast_episodes,
    get_recent_episodes
)

# Allow nested event loops in Jupyter
nest_asyncio.apply()

async def run_db_operation(operation):
    """Run a database operation in an async context"""
    async with get_db() as db:
        return await operation(db)

def run_async(coro):
    """Run an async operation from a sync context"""
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    return loop.run_until_complete(coro)

### Helper Function for Async Operations

In [11]:
async def run_db_operation(operation):
    """Run a database operation in an async context"""
    async with get_db() as db:
        return await operation(db)

def run_async(coro):
    """Run an async operation from a sync context"""
    return asyncio.get_event_loop().run_until_complete(coro)

## Podcast Operations

### List All Podcasts

In [38]:
podcasts = run_async(run_db_operation(lambda db: list_podcasts(db, with_episode_count=True)))
df = pd.DataFrame(podcasts)
df

,_sa_instance_state,description,id,image_url,frequency,publisher,name,rss_url,created_at,tags,episode_count
0,<sqlalchemy.orm.state.InstanceState object at ...,"After 25 years at the Late Night desk, Conan r...",07124837-ca0b-4db9-8abb-38f3c7269c92,https://placehold.co/400,2.18,Team Coco & Earwolf,Conan O'Brien Needs a Friend,https://feeds.simplecast.com/dHoohVNH,2025-01-31 00:14:10.199132+00:00,None,0
1,<sqlalchemy.orm.state.InstanceState object at ...,"Industry veterans, degenerate gamblers & besti...",457c609c-2015-472e-b64a-13bea03a645c,https://placehold.co/400,1.01,"All-In Podcast, LLC",All-In,https://allinchamathjason.libsyn.com/rss,2025-01-31 00:14:10.007078+00:00,None,0
2,<sqlalchemy.orm.state.InstanceState object at ...,Football’s funniest family duo — Jason Kelce f...,4be70d05-d1ed-40af-8c8b-466be83fbc5b,https://placehold.co/400,0.95,Wondery,New Heights With Jason & Travis Kelce,https://rss.art19.com/new-heights,2025-01-31 00:14:11.209835+00:00,None,0
3,<sqlalchemy.orm.state.InstanceState object at ...,The official podcast of comedian Joe Rogan.,6a61382f-f9e6-4023-9862-7aa1b3438263,https://placehold.co/400,3.4,Joe Rogan,The Joe Rogan Experience,https://feeds.megaphone.fm/GLT1412515089,2025-01-31 00:14:10.806094+00:00,None,1
4,<sqlalchemy.orm.state.InstanceState object at ...,What happened this past weekend. And sometimes...,862a7d48-5bae-4b79-866b-ddb4719fe144,https://placehold.co/400,1.34,Theo Von,This Past Weekend w/ Theo Von,https://feeds.megaphone.fm/thispastweekend,2025-01-31 00:14:11.632424+00:00,None,0
5,<sqlalchemy.orm.state.InstanceState object at ...,“Hard Fork” is a show about the future that’s ...,941b4ac1-cee1-4060-92ad-384f33bad329,https://placehold.co/400,1.0,The New York Times,Hard Fork,https://feeds.simplecast.com/6HKOhNgS,2025-01-31 00:14:10.601821+00:00,None,0
6,<sqlalchemy.orm.state.InstanceState object at ...,This is what the news should sound like. The b...,a150e573-0086-4a14-8c53-6a5da1f815f0,https://placehold.co/400,1.0,The New York Times,The Daily,https://feeds.simplecast.com/Sl5CSM3S,2025-01-31 00:14:10.293618+00:00,None,0
7,<sqlalchemy.orm.state.InstanceState object at ...,You can change your life and Mel Robbins will ...,a45a9899-f7e4-475d-b430-c2c76ae4dce7,https://placehold.co/400,2.28,Mel Robbins,The Mel Robbins Podcast,https://feeds.simplecast.com/UCwaTX1J,2025-01-31 00:14:11.109873+00:00,None,0
8,<sqlalchemy.orm.state.InstanceState object at ...,The most-listened to podcast by women. Alex Co...,bbac8fbb-05e9-4089-9666-abc3012b90ce,https://placehold.co/400,2.18,Alex Cooper,Call Her Daddy,https://feeds.simplecast.com/mKn_QmLS,2025-01-31 00:14:10.102062+00:00,None,0
9,<sqlalchemy.orm.state.InstanceState object at ...,"Fresh Air from WHYY, the Peabody Award-winning...",bcf7a02a-a19d-4e8f-a778-0fd021b412d1,https://placehold.co/400,6.08,NPR,Fresh Air,https://feeds.npr.org/381444908/podcast.xml,2025-01-31 00:14:10.501789+00:00,None,0


In [34]:
df['category'] = df['tags'].str[0]
df['tags'] = pd.NA
df_local = df.copy()
df

,_sa_instance_state,description,id,image_url,frequency,publisher,name,rss_url,created_at,tags,episode_count,category
0,<sqlalchemy.orm.state.InstanceState object at ...,“Hard Fork” is a show about the future that’s ...,941b4ac1-cee1-4060-92ad-384f33bad329,https://placehold.co/400,1.0,The New York Times,Hard Fork,https://feeds.simplecast.com/6HKOhNgS,2025-01-31 00:14:10.601821+00:00,<NA>,0,banter
1,<sqlalchemy.orm.state.InstanceState object at ...,"Conversations about science, technology, histo...",d4f7a71c-a867-4327-a455-6b52d691b334,https://placehold.co/400,0.97,Lex Fridman,Lex Fridman Podcast,https://lexfridman.com/feed/podcast/,2025-01-31 00:14:10.902759+00:00,<NA>,0,interview
2,<sqlalchemy.orm.state.InstanceState object at ...,"Marc Maron welcomes comedians, actors, directo...",f50345ae-5fff-497b-b6db-a0eea178f264,https://placehold.co/400,2.06,Marc Maron,WTF with Marc Maron,https://access.acast.com/rss/62a222737c0214001...,2025-01-31 00:14:11.851301+00:00,<NA>,0,interview
3,<sqlalchemy.orm.state.InstanceState object at ...,A weekly podcast about the intersection betwee...,fdc7b6c0-efce-4df1-8cfb-f09ac78e2165,https://placehold.co/400,1.48,The Overhead Wire,Talking Headways,https://rss.pdrl.fm/6cddbc/streetsblog.libsyn....,2025-01-31 00:14:11.513409+00:00,<NA>,0,interview
4,<sqlalchemy.orm.state.InstanceState object at ...,"Each Tuesday and Friday, Ezra Klein invites yo...",eb09c465-3f30-4111-b61c-9822a951c77c,https://placehold.co/400,2.0,New York Times Opinion,The Ezra Klein Show,https://feeds.simplecast.com/kEKXbjuJ,2025-01-31 00:14:10.406996+00:00,<NA>,0,interview
5,<sqlalchemy.orm.state.InstanceState object at ...,This is what the news should sound like. The b...,a150e573-0086-4a14-8c53-6a5da1f815f0,https://placehold.co/400,1.0,The New York Times,The Daily,https://feeds.simplecast.com/Sl5CSM3S,2025-01-31 00:14:10.293618+00:00,<NA>,0,interview
6,<sqlalchemy.orm.state.InstanceState object at ...,The official podcast of comedian Joe Rogan.,6a61382f-f9e6-4023-9862-7aa1b3438263,https://placehold.co/400,3.4,Joe Rogan,The Joe Rogan Experience,https://feeds.megaphone.fm/GLT1412515089,2025-01-31 00:14:10.806094+00:00,<NA>,1,interview
7,<sqlalchemy.orm.state.InstanceState object at ...,"On Mondays, Jon Stewart hosts The Daily Show, ...",e83e4e91-b337-4d28-abcd-e9a89f6d0d5b,https://placehold.co/400,0.74,Comedy Central,The Weekly Show with Jon Stewart,https://feeds.megaphone.fm/BVLLC2163264914,2025-01-31 00:14:11.742513+00:00,<NA>,0,interview
8,<sqlalchemy.orm.state.InstanceState object at ...,The most-listened to podcast by women. Alex Co...,bbac8fbb-05e9-4089-9666-abc3012b90ce,https://placehold.co/400,2.18,Alex Cooper,Call Her Daddy,https://feeds.simplecast.com/mKn_QmLS,2025-01-31 00:14:10.102062+00:00,<NA>,0,interview
9,<sqlalchemy.orm.state.InstanceState object at ...,Football’s funniest family duo — Jason Kelce f...,4be70d05-d1ed-40af-8c8b-466be83fbc5b,https://placehold.co/400,0.95,Wondery,New Heights With Jason & Travis Kelce,https://rss.art19.com/new-heights,2025-01-31 00:14:11.209835+00:00,<NA>,0,banter


In [ ]:
import pandas as pd
import asyncio
from datetime import datetime
import pytz
from sqlalchemy import text

async def update_podcasts_from_df(db, df):
    """
    Update podcast records from a pandas DataFrame.
    The DataFrame should have columns matching the Podcast model fields.
    """
    print("Starting update process...")
    updated_count = 0
    
    # Define valid columns (exclude SQLAlchemy internal attributes)
    valid_columns = [
        'name', 'publisher', 'description', 'rss_url', 
        'image_url', 'tags', 'frequency', 'created_at'
    ]
    
    # Iterate through DataFrame rows
    for _, row in df.iterrows():
        # Convert row to dict, dropping any NaN values and filtering valid columns
        podcast_data = {
            k: v for k, v in row.dropna().to_dict().items() 
            if k in valid_columns
        }
        
        # Get existing podcast by RSS URL
        existing_podcast = await get_podcast_by_rss_url(db, row['rss_url'])
        
        if existing_podcast:
            # Update existing podcast
            if podcast_data:  # Only update if we have valid data
                await db.execute(
                    text(f"""
                        UPDATE podcasts 
                        SET {', '.join(f"{k} = :{k}" for k in podcast_data.keys())}
                        WHERE id = :id
                    """),
                    {**podcast_data, "id": existing_podcast.id}
                )
                print(f"Updated {row.get('name', 'Unknown podcast')}")
                updated_count += 1
        else:
            # Create new podcast
            new_podcast = await create_podcast(db, podcast_data)
            print(f"Created {row.get('name', 'Unknown podcast')}")
            updated_count += 1
    
    # Commit all changes
    await db.commit()
    print(f"\nUpdate completed! Modified {updated_count} podcasts.")

# Get the existing podcasts data
podcasts = run_async(run_db_operation(lambda db: list_podcasts(db, with_episode_count=True)))
df = pd.DataFrame(podcasts)

# Run the update
try:
    print("Starting update process...")
    result = run_async(run_db_operation(lambda db: update_podcasts_from_df(db, df)))
    print("Update process finished!")
except Exception as e:
    print(f"Error during update: {str(e)}")

Starting update process...
Starting update process...
Error during update: (sqlalchemy.dialects.postgresql.asyncpg.ProgrammingError) <class 'asyncpg.exceptions.UndefinedColumnError'>: column "_sa_instance_state" of relation "podcasts" does not exist
[SQL: 
                    UPDATE podcasts 
                    SET _sa_instance_state = $1, description = $2, id = $3, image_url = $4, frequency = $5, publisher = $6, name = $7, rss_url = $8, created_at = $9
                    WHERE id = $3
                ]
[parameters: (<sqlalchemy.orm.state.InstanceState object at 0x11f37a930>, '“Hard Fork” is a show about the future that’s already here. Each week, journalists Kevin Roose and Casey Newton explore and make sense of the latest in the rapidly changing world of tech.', UUID('941b4ac1-cee1-4060-92ad-384f33bad329'), 'https://placehold.co/400', '1.0', 'The New York Times', 'Hard Fork', 'https://feeds.simplecast.com/6HKOhNgS', Timestamp('2025-01-31 00:14:10.601821+0000', tz='UTC'))]
(Backgroun

### Create New Podcast

In [ ]:
new_podcast_data = {
    "name": "My Test Podcast",
    "publisher": "Test Publisher", 
    "description": "A test podcast",
    "rss_url": "https://test.com/feed",
    "image_url": "https://test.com/image.png",
    "tags": ["test", "demo"],
    "prompt_addition": None,
    "episode_count": 0
}

async def create_new_podcast(db, data):
    podcast = await create_podcast(db, data)
    await db.commit()
    return podcast

new_podcast = run_async(run_db_operation(lambda db: create_new_podcast(db, new_podcast_data)))
print(f"Created: {new_podcast.name} (ID: {new_podcast.id})")

## Episode Operations

### Get Recent Episodes

In [ ]:
recent = run_async(run_db_operation(lambda db: get_recent_episodes(db, limit=5)))

for ep in recent:
    print(f"\nEpisode: {ep.title}")
    print(f"Podcast: {ep.podcast.name}")
    print(f"Published: {ep.publish_date}")
    if ep.summary:
        print(f"Summary: {ep.summary[:100]}...")

### Get Episodes for a Specific Podcast

In [ ]:
# Replace with your podcast ID
PODCAST_ID = "your-podcast-id-here"

episodes = run_async(run_db_operation(lambda db: get_podcast_episodes(db, PODCAST_ID)))

for ep in episodes:
    print(f"\nEpisode: {ep.title}")
    print(f"Published: {ep.publish_date}")
    print(f"GUID: {ep.rss_guid}")

### Create New Episode

In [ ]:
new_episode_data = {
    "podcast_id": "your-podcast-id-here",  # Replace with actual ID
    "rss_guid": "unique-guid-here",
    "title": "Test Episode",
    "publish_date": datetime.now(timezone.utc),
    "summary": "A test episode"
}

async def create_new_episode(db, data):
    episode = await create_episode(db, data)
    await db.commit()
    return episode

new_episode = run_async(run_db_operation(lambda db: create_new_episode(db, new_episode_data)))
print(f"Created: {new_episode.title} (ID: {new_episode.id})")

### Find Episode by GUID

In [ ]:
# Replace with your episode GUID
EPISODE_GUID = "your-guid-here"

episode = run_async(run_db_operation(lambda db: get_episode_by_guid(db, EPISODE_GUID)))
if episode:
    print(f"Found: {episode.title}")
    print(f"Podcast ID: {episode.podcast_id}")
    print(f"Published: {episode.publish_date}")
else:
    print("Episode not found")